# Phase 3: Machine Learning

In this notebook, we will:

- Load the cleaned dataset
- Split the data into train and test sets
- Train and evaluate baseline models
- (Optionally) Improve model performance


In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.makedirs("images", exist_ok=True)

In [3]:
df = pd.read_csv("../data/inputs/cleaned/cleaned_credit_data.csv")
df.head()

,limit_bal,sex,education,marriage,age,pay_delay_sep,pay_delay_aug,pay_delay_jul,pay_delay_jun,pay_delay_may,...,payment_amount_may,payment_amount_apr,default,total_bill_amount,total_payment_amount,payment_to_bill_ratio,payment_delay_count,high_utilization,avg_payment_delay,has_ever_paid_on_time
0,20000.0,female,university,married,24,2,2,-1,-1,-2,...,0.0,0.0,1,7704.0,689.0,0.089434,2,0,-0.333333,1
1,120000.0,female,university,single,26,-1,2,0,0,0,...,0.0,2000.0,1,17077.0,5000.0,0.292791,2,0,0.500000,1
2,90000.0,female,university,single,34,0,0,0,0,0,...,1000.0,5000.0,0,101653.0,11018.0,0.108388,0,0,0.000000,0
3,50000.0,female,university,married,37,0,0,0,0,0,...,1069.0,1000.0,0,231334.0,8388.0,0.036259,0,1,0.000000,0
4,50000.0,male,university,married,57,-1,0,-1,0,0,...,689.0,679.0,0,109339.0,59049.0,0.540054,0,0,-0.333333,1
